In [1]:
from heapq import *

def find_employee_free_time(schedule):
    n = len(schedule)
    result = []
    if schedule is None or n == 0:
        return result

    minHeap = []
    # insert the first interval of each employee to the queue
    for i in range(n):
        heappush(minHeap, EmployeeInterval(schedule[i][0], i, 0))

    previousInterval = minHeap[0].interval
    while minHeap:
        queueTop = heappop(minHeap)
        # if previousInterval is not overlapping with the next interval, insert a free interval
        if previousInterval.end < queueTop.interval.start:
            result.append(Interval(previousInterval.end,
                                   queueTop.interval.start))
            previousInterval = queueTop.interval
        else:  # overlapping intervals, update the previousInterval if needed
            if previousInterval.end < queueTop.interval.end:
                previousInterval = queueTop.interval

        # if there are more intervals available for the same employee, add their next interval
        employeeSchedule = schedule[queueTop.employeeIndex]
        if len(employeeSchedule) > queueTop.intervalIndex + 1:
            heappush(minHeap, EmployeeInterval(employeeSchedule[queueTop.intervalIndex + 1], queueTop.employeeIndex,
                                               queueTop.intervalIndex + 1))

    return result

class Interval:
    def __init__(self, start, end):
        self.start = start
        self.end = end

    def print_interval(self):
        print("[" + str(self.start) + ", " + str(self.end) + "]", end='')

class EmployeeInterval:

    def __init__(self, interval, employeeIndex, intervalIndex):
        self.interval = interval  # interval representing employee's working hours
        # index of the list containing working hours of this employee
        self.employeeIndex = employeeIndex
        self.intervalIndex = intervalIndex  # index of the interval in the employee list

    def __lt__(self, other):
        # min heap based on meeting.end
        return self.interval.start < other.interval.start
        
def test():

    input = [[Interval(1, 3), Interval(5, 6)], [
        Interval(2, 3), Interval(6, 8)]]
    print("Free intervals: ", end='')
    for interval in find_employee_free_time(input):
        interval.print_interval()
    print()

    input = [[Interval(1, 3), Interval(9, 12)], [
        Interval(2, 4)], [Interval(6, 8)]]
    print("Free intervals: ", end='')
    for interval in find_employee_free_time(input):
        interval.print_interval()
    print()

    input = [[Interval(1, 3)], [
        Interval(2, 4)], [Interval(3, 5), Interval(7, 9)]]
    print("Free intervals: ", end='')
    for interval in find_employee_free_time(input):
        interval.print_interval()
    print()

test()

Free intervals: [3, 5]
Free intervals: [4, 6][8, 9]
Free intervals: [5, 7]


In [1]:
def merge(intervals_a, intervals_b):
    i, j = 0, 0
    start_idx, end_idx = 0, 1
    result = list()
    
    while i < len(intervals_a) and j < len(intervals_b):
        
        # check if a overlaps b
        a_overlaps_b = intervals_a[i][start_idx] >= intervals_b[j][start_idx] and \
            intervals_a[i][start_idx] <= intervals_b[j][end_idx]
        
        # check if b overlaps a
        
        b_overlaps_a = intervals_b[j][start_idx] >= intervals_a[i][start_idx] and \
            intervals_b[j][start_idx] <= intervals_a[j][end_idx]
        
        if a_overlaps_b or b_overlaps_a:
            result.append([max(intervals_a[i][start_idx], intervals_b[j][start_idx]), 
                           min(intervals_a[i][end_idx], intervals_b[j][end_idx])])
            
        if intervals_a[i][end_idx] < intervals_b[j][end_idx]:
            i += 1
        else:
            j += 1
            
    return result

interval_dict = {'BLAH': [[4, 6]], 'NOS': [[1, 8]]}
interval_a = interval_dict['BLAH']
interval_b = interval_dict['NOS']
merge(interval_a, interval_b)

[[4, 6]]

In [5]:
def getFreeTimes(schedule):
    
    times = [(start_time, end_time) for item in schedule for start_time, end_time in item]
    times.sort(key=lambda a: a[0])
    
    prev_start_time, prev_end_time = times[0][0], times[0][1]
    
    results = list()
    
    cur_end_time_max = prev_end_time
    
    for idx in range(1, len(times)):
        current_time = times[idx]
        start_time, end_time = current_time
        
        if start_time - prev_end_time > 0 and cur_end_time_max < start_time:
            results.append((cur_end_time_max, start_time))
        
        prev_start_time = start_time
        prev_end_time = end_time
        cur_end_time_max = max(cur_end_time_max, end_time)
        
    return results

schedule = [
  [
    (2, 3),
    (6, 7)
  ],
  [
    (0, 1),
    (5, 6)
  ],
  [
    (1, 2),
    (6, 7),
    (8, 9)
  ]
]
getFreeTimes(schedule)

[(3, 5), (7, 8)]

In [3]:
schedule = [
  [
    (2, 3),
    (4, 5)
  ],
  [
    (1, 2),
    (2, 3)
  ]
]
getFreeTimes(schedule)

[(3, 4)]

In [17]:
# How do you add intervals and merge them for a large stream of intervals?
from typing import List


class TreeNode:
    def __init__(self, start, end, middle):
        self.start = start
        self.end = end
        self.middle = middle
        self.left = self.right = None

        
class IntervalStream:
    def __init__(self):
        self.root = None
    
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        if not intervals:
            return []
        
        for start, end in intervals:
            if not self.root:
                self.root = TreeNode(start, end, (start + end) // 2)
            else:
                self.add(self.root, start, end)
        
        return self.query(self.root)
    
    
    def add(self, node, start, end):     
        if end < node.middle:
            if node.left:
                self.add(node.left, start, end)
            else:
                node.left = TreeNode(start, end, (start + end) // 2)
        
        elif start > node.middle:
            if node.right:
                self.add(node.right, start, end)
            else:
                node.right = TreeNode(start, end, (start + end) // 2)
        
        else:
            node.start = min(node.start, start)
            node.end = max(node.end, end)
    
    def query(self, node):
        if not node:
            return []
        
        # merge-sort divide and conquer
        left_intervals = self.query(node.left)
        right_intervals = self.query(node.right)
        res = []
        
        inserted = False
        print((left_intervals, right_intervals))
        for lres in left_intervals:
            if lres[1] < node.start:
                res.append(lres)
            else:
                res.append([min(lres[0], node.start), node.end])
                inserted = True
                break
        
        if not inserted:
            res.append([node.start, node.end])
        
        for rres in right_intervals:
            if rres[0] <= node.end:
                res[-1][1] = max(node.end, rres[1])
            else:
                res.append(rres)
        
        return res
    
stream = IntervalStream()
stream.merge([[1,2],[2,3],[5,9],[7,8]])
stream.query(TreeNode(7,20,15//2))

([], [])
([], [[5, 9]])
([], [[2, 3], [5, 9]])
([], [])


[[7, 20]]

In [69]:
class IntervalStream:
    def __init__(self):
        self.intervals = list()
        
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        # sort by starting time
        # compare prior end time with start time

        # [1,3], [3,4], [4,5] -> [1,5]


        if not intervals:
            return list()


        intervals.sort(key=lambda interval: interval[0])

        merged_intervals = list()

        for interval in intervals:

            if not merged_intervals or interval[0] > merged_intervals[-1][1]:
                merged_intervals.append(interval)
            else:
                merged_intervals[-1][1] = max(merged_intervals[-1][1], interval[1])
                
        self.intervals.extend(merged_intervals)
        return merged_intervals
    
    def add(self, interval: List[int]):
        # binary search for values since the property of merged intervals are all isolated
        print(f'adding {interval}')
        start, end = interval
        
        left, right = 0, len(self.intervals)-1
        while left <= right:
            midpoint = (left + right) // 2
            mid_interval = self.intervals[midpoint]
            
            mid_start = mid_interval[0]
            
            if start < mid_start:
                right = midpoint-1
            elif start > mid_start:
                left = midpoint + 1
            else:
                # if there is a match here, probably a merge point
                self.intervals[midpoint][1] = max(self.intervals[midpoint][1], end)
                return
        
        
        if right > left or right < left:
            self.intervals.insert(left, [start, end])
            return
            
        #print(f'left: {left} right: {right}')
        base_start, base_end = self.intervals[right]
        print('start')
        print((base_start, base_end))
        print((start, end))
        print('end')
        if start < base_end:
            print('here')
            self.intervals[right][1] = max(self.intervals[right][1], end)
            print('out')
        else:
            print('last')
            self.intervals.insert(left, [start, end])
            
        print('exit')
        
        
stream = IntervalStream()
stream.merge([[1,2],[2,3],[5,9],[7,8]])
print(stream.intervals)
stream.add([12,13])
print(stream.intervals)
stream.add([1,4])
print(stream.intervals)
stream.add([6,11])
print(stream.intervals)
stream.add([14,20])
print(stream.intervals)
stream.add([0,1])
print(stream.intervals)
# [1,4],[5,11],[12,13],[14,20]

[[1, 3], [5, 9]]
adding [12, 13]
[[1, 3], [5, 9], [12, 13]]
adding [1, 4]
[[1, 4], [5, 9], [12, 13]]
adding [6, 11]
[[1, 4], [5, 9], [6, 11], [12, 13]]
adding [14, 20]
[[1, 4], [5, 9], [6, 11], [12, 13], [14, 20]]
adding [0, 1]
[[0, 1], [1, 4], [5, 9], [6, 11], [12, 13], [14, 20]]
